<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/030_disbalance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://kaggle.com/kernels/welcome?src=https://github.com/gurovic/MLCourse/blob/main/030_disbalance.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>

# Дисбаланс классов

In [ ]:
!pip install pandas numpy scikit-learn imbalanced-learn tensorflow

## 🟢 Базовый уровень (Основные подходы)

### 1.1 Понимание дисбаланса

In [ ]:
from sklearn.datasets import make_classification

# Генерация несбалансированных данных
X, y = make_classification(
    n_samples=1000,
    weights=[0.95, 0.05],  # 95% negative class
    random_state=42
)

# Анализ
print("Распределение классов:", {0: (y == 0).sum(), 1: (y == 1).sum()})

### 1.2 Случайное передискретизирование

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# oversampling - увеличиваем количество объектов в маленьких классах
oversampler = RandomOverSampler(sampling_strategy=0.5, random_state=42)
X_over, y_over = oversampler.fit_resample(X, y)

print("Распределение классов после oversampling:", {0: (y_over == 0).sum(), 1: (y_over == 1).sum()})

# undersampling - уменьшаем количество объектов в больших классах
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_under, y_under = undersampler.fit_resample(X, y)

print("Распределение классов после undersampling:", {0: (y_over == 0).sum(), 1: (y_over == 1).sum()})

### 1.3 Взвешивание классов

In [ ]:
from sklearn.linear_model import LogisticRegression

# Автоматическое взвешивание
model = LogisticRegression(class_weight='balanced')

# Ручное задание весов
weights = {0: 1, 1: 10}  # Увеличиваем вес миноритарного класса
model = LogisticRegression(class_weight=weights)

## 🟡 Продвинутый уровень (SMOTE и ансамбли)

### 2.1 SMOTE (Synthetic Minority Oversampling)

In [ ]:
from imblearn.over_sampling import SMOTE

# Создание синтетических примеров
smote = SMOTE(
    sampling_strategy=0.3,
    k_neighbors=5,
    random_state=42
)
X_smote, y_smote = smote.fit_resample(X, y)

import matplotlib.pyplot as plt
import numpy as np

# Построение облака точек
plt.figure(figsize=(10, 6))
plt.scatter(X[:,0], y, color='blue', alpha=0.7, label='original')
plt.scatter(X_smote[:,0], y_smote+0.01, color='red', alpha=0.7, label='smote, shifted up')
plt.title('Облако точек (X[:,0], y)')
plt.xlabel('X')
plt.ylabel('y')
plt.grid(True)
plt.legend()
plt.show()

### 2.2 Ансамблевые методы

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier

# Модель с балансировкой
brf = BalancedRandomForestClassifier(
    n_estimators=100,
    sampling_strategy=0.5,
    replacement=True,
    random_state=42
)
brf.fit(X, y)

### 2.3 Метрики для оценки

In [ ]:
from sklearn.metrics import classification_report

# Использование F1 вместо accuracy
model.fit(X, y)
y_pred = model.predict(X)
print(classification_report(y, y_pred, target_names=['Class 0', 'Class 1']))

## 🔴 Экспертный уровень (Продвинутые техники)

### 3.1 Генеративные модели (GAN)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Генератор для создания синтетических данных
class Generator(nn.Module):
    def __init__(self, input_dim=10, output_dim=1):  # output_dim зависит от размерности X
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
            nn.Tanh()  # Для нормализации выхода в диапазон [-1, 1]
        )

    def forward(self, x):
        return self.model(x)

# Пример использования генератора
def build_generator(input_dim=10, output_dim=1):
    return Generator(input_dim, output_dim)

# Предположим, у вас есть данные X
# X = np.random.randn(1000, ...)  # Пример данных (замените на реальные)
output_dim = 10  # Замените на размер признаков вашего датасета

# Создаем генератор
generator = build_generator(input_dim=10, output_dim=output_dim)

# Пример генерации случайного батча из шума
batch_size = 64
noise = torch.randn(batch_size, 10)  # Случайный шум размером (64, 10)
generated_data = generator(noise)

print("Сгенерированные данные:", generated_data.shape)
# Вывод: torch.Size([64, output_dim])

### 3.2 Динамическое взвешивание классов

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DynamicWeightedLoss(nn.Module):
    """
    Кастомная функция потерь с динамическими весами для балансировки классов.

    Параметры:
        alpha (float): Вес для отрицательного класса.
                       Чем выше alpha, тем больше штраф за ошибки по негативному классу.
    """
    def __init__(self, alpha=0.5):
        super(DynamicWeightedLoss, self).__init__()
        self.alpha = alpha  # Управление балансом между классами

    def forward(self, y_pred, y_true):
        # Преобразуем y_pred в вероятности через sigmoid
        y_pred = torch.sigmoid(y_pred)

        # Маска для положительных и отрицательных классов
        pos_mask = (y_true == 1).float()
        neg_mask = (y_true == 0).float()

        # Подсчет количества примеров каждого класса
        num_pos = torch.sum(pos_mask)
        num_neg = torch.sum(neg_mask)

        # Вычисление весов (избегаем деления на ноль)
        pos_weight = (1 - self.alpha) / (num_pos + 1e-8)
        neg_weight = self.alpha / (num_neg + 1e-8)

        # Формируем веса для каждой точки
        weights = pos_weight * pos_mask + neg_weight * neg_mask

        # Бинарная кросс-энтропия
        bce = F.binary_cross_entropy(y_pred, y_true, reduction='none')

        # Взвешенная средняя потеря
        weighted_loss = torch.mean(weights * bce)

        return weighted_loss

In [ ]:
#Пример использования

# Пример данных
y_true = torch.tensor([0, 1, 0, 1], dtype=torch.float32)   # Реальные метки
y_pred = torch.tensor([0.2, 0.7, 0.4, 0.8], dtype=torch.float32)  # Предсказанные логиты или вероятности

# Инициализация функции потерь
loss_fn = DynamicWeightedLoss(alpha=0.7)  # Больше внимания негативному классу

# Вычисление потерь
loss = loss_fn(y_pred, y_true)
print("Custom Loss:", loss.item())

### 3.3 Оптимизация порога классификации

In [ ]:
from sklearn.metrics import precision_recall_curve
import numpy as np

# Пример данных (замените на свои)
y_pred = np.array([0, 1, 0, 1, 0, 1, 0, 1])  # Истинные метки
y_probs = np.array([0.1, 0.8, 0.2, 0.95, 0.4, 0.7, 0.3, 0.9])  # Предсказанные вероятности

# Поиск оптимального порога
precisions, recalls, thresholds = precision_recall_curve(y_pred, y_probs)

# Рассчитываем F1-меру для всех порогов
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-8)

# Находим порог с максимальным F1
best_threshold = thresholds[np.argmax(f1_scores)]

# Применяем порог к вероятностям
y_pred_custom = (y_probs >= best_threshold).astype(int)

print("Истинные метки", y_pred)
print("Предсказанные вероятности", y_probs)
print("Предсказанные метки", y_pred_custom)

## 📊 Чеклист по уровням

| Уровень | Навыки |
|---------|--------|
| 🟢 | RandomOverSampler, class_weight, анализ распределения |
| 🟡 | SMOTE, BalancedRandomForest, метрики F1/ROC-AUC |
| 🔴 | GAN-синтез, кастомные функции потерь, оптимизация порога |

## ⚠️ Антипаттерны
### Для всех уровней:
- **Использование accuracy как метрики** для несбалансированных данных
- **Слепое применение SMOTE** без анализа природы данных
- **Полное устранение дисбаланса** (может ухудшить качество)

### 🔴 Эксперты:
- **Переобучение на синтетических данных** (тестируйте на реальных данных)
- **Игнорирование costs-sensitive анализа** (разная цена ошибок)

## 🚀 Советы

In [ ]:
# Установите библиотеку, если ещё не установлено
# !pip install yellowbrick

from yellowbrick.classifier import ClassBalance
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Создаем синтетический несбалансированный датасет
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2,
                           weights=[0.9, 0.1], random_state=42)

# Разделяем на train и test
y = y.astype(int)  # Убедиться, что метки целочисленные
y_train, y_test = train_test_split(y, test_size=0.3, stratify=y, random_state=42)

# Визуализация баланса классов
visualizer = ClassBalance(labels=['Class 0', 'Class 1'])
visualizer.fit(y_train, y_test)
visualizer.show()

In [ ]:
# Комбинируйте методы
from imblearn.pipeline import Pipeline

pipeline = Pipeline([
    ('smote', SMOTE(sampling_strategy=0.3)),
    ('undersample', RandomUnderSampler(sampling_strategy=0.5)),
    ('model', LogisticRegression())
])

In [ ]:
# Анализ ошибок через матрицу
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y_pred, y_pred_custom, normalize='true')

## 📌 Тренировочные задания

### 🟢 Базовый уровень
**Задача 1:** Примените RandomUnderSampler к датасету кредитного мошенничества (`fraud_detection.csv`). Сравните F1-score до/после.

### 🟡 Продвинутый уровень
**Задача 2:** Используя SMOTE, сбалансируйте классы в датасете медицинских диагнозов. Постройте ROC-кривые для моделей `LogisticRegression` и `BalancedRandomForest`.

### 🔴 Экспертный уровень
**Задача 3:** Продемонстрируйте полный цикл ML, включающий работу с дисбалансом классов, на примере задачи fraud detection.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler

# -----------------------------
# 1. Создание синтетических данных
# -----------------------------
np.random.seed(42)

n_samples = 10000
data = {
    'amount': np.random.lognormal(mean=3, sigma=1, size=n_samples),
    'time_hours': np.random.randint(0, 24 * 7, size=n_samples),
    'user_age': np.random.randint(18, 80, size=n_samples),
    'location_mismatch': np.random.choice([0, 1], size=n_samples, p=[0.9, 0.1]),
    'device_hash_known': np.random.choice([0, 1], size=n_samples, p=[0.7, 0.3]),
    'ip_address_risky': np.random.choice([0, 1], size=n_samples, p=[0.95, 0.05]),
    'is_fraud': np.random.choice([0, 1], size=n_samples, p=[0.99, 0.01])
}

df = pd.DataFrame(data)

# -----------------------------
# 2. Разделение на признаки и целевую переменную
# -----------------------------
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

# -----------------------------
# 3. Разделение на обучающую и тестовую выборки
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# -----------------------------
# 4. Применение Undersampling
# -----------------------------
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# -----------------------------
# 5. Обучение модели
# -----------------------------
model = RandomForestClassifier(random_state=42)  # Можно попробовать: LogisticRegression(), XGBClassifier()
model.fit(X_resampled, y_resampled)

# -----------------------------
# 6. Предсказания и оценка
# -----------------------------
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("Classification Report:\n", classification_report(y_test, y_pred))

# -----------------------------
# 7. Confusion Matrix
# -----------------------------
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# -----------------------------
# 8. ROC Curve & AUC Score
# -----------------------------
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

# -----------------------------
# 9. Визуализация баланса классов
# -----------------------------
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.countplot(x=y_resampled, ax=axes[0])
axes[0].set_title('После Undersampling')

sns.countplot(x=y_train, ax=axes[1])
axes[1].set_title('До Undersampling')

plt.tight_layout()
plt.show()

# -----------------------------
# 10. Feature Importance с SHAP
# -----------------------------
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test, plot_type="bar", feature_names=X.columns)

## 📌 Заключение
Ключевые идеи:
1. **Не всегда нужно балансировать классы** — зависит от задачи.
2. **Сочетайте методы** (например, SMOTE + Undersampling).
3. **Экспериментируйте с метриками** — Precision/Recall Tradeoff.
4. **Учитывайте стоимость ошибок** (ложные положительные vs. отрицательные).